In [ ]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn as nn

In [ ]:
class GNN(torch.nn.module):

    def __init__ (self, input_channels, hidden_channels, output_channels):
        self.conv1 = GCNConv(input_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, output_channels)
 

    def model(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.log_softmax(x, dim=1 )
        return x
    

model = GNN(in_channels, hidden_channels = 64, out_channels)

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

def train(model, optimiser, data):
    model.train()
    optimiser.zero_grad()
    out = model(data.x, data.edge_index)
    loss = (out[data.train_mask], data.y[data.train_mask].squeeze()) 
    loss.backward()
    optimiser.step()
    return loss.item()

def validate (model, data):
    model.eval()
    with torch.no_grad():
        out =model(data.x, data.edge_index) 
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask].squeeze())
    return val_loss.item()

train_loss = []
val_loss = []

for epoch in range(200):
    train_loss.append(train(model, optimiser, subset_graph))
    val_loss.append(validate(model, subset_graph)) 


In [ ]:
class Transformer(torch.nn.module):
    def __init__(self, embed_dim, num_heads, num_layers):
        self.trans = nn.TransformerEncoderLayer(d_model=embed_dim,
            nhead=num_heads,
            batch_first=True)


class SpatioTemporalBlock(torch.nn.module):
    def __init__(self, in_channels, hidden_channels, num_heads):
        self.spatialBlock = GNN(in_channels, hidden_channels)
        self.temporalBlock = Transformer(hidden_channels, num_heads, 1)

class Model(torch.nn.module):
    def __init__(self, num_blocks, in_channels, hidden_channels, num_heads):
        self.